# LOAD Modules

In [1]:
%matplotlib inline
#%matplotlib widget
%load_ext autoreload
%autoreload 2


import numpy as np
from visualisations import eeg_sample_plot, events_distribution_plot
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import DataLoader
from datasets import ChannelsVoltageDataset
from neural_nets.nn_models_getter import get_nn_model
from optimizers import get_optimizer
from utils_train import fit, test, plot_all_metrics, plot_val_metrics
from configs import configs_ozhan, configs_tim, configs_joaquin
from data_loader_creation import get_dataloader_objects
from classification_results import results_storer



# Select the configuration

In [2]:
""" USER: SELECT THE CONFIGURATION YOU NEED """
#myList = configs_tim.list_of_configs
#myList = configs_joaquin.list_of_configs
myList = configs_ozhan.list_of_configs

# Train

In [ ]:
training_curves = {}

for my_cfg in myList:
    
    curve_name = my_cfg.nn_list[my_cfg.nn_selection_idx] + "_" + my_cfg.optimizer_list[my_cfg.optimizer_selection_idx]
    print(curve_name,"\n-------------------------\n", flush=True)
    
    """ PREPARE DATALOADERS """
    train_dl, val_dl, test_dl, input_dimension_, output_dimension_ = get_dataloader_objects(my_cfg)


    """CLASSIFICATION"""
    # Get the model
    model_untrained = get_nn_model(my_cfg.nn_list[my_cfg.nn_selection_idx], input_dimension=input_dimension_,
                                   output_dimension=output_dimension_, dropout=my_cfg.dropout_perc)

    # Get the optimizer
    optimizer = get_optimizer(my_cfg.optimizer_list[my_cfg.optimizer_selection_idx], my_cfg.learning_rate,
                              model_untrained.parameters(), my_cfg.momentum, my_cfg.weight_decay)

    # Train and show validation loss
    train_losses, train_accuracies, val_losses, val_accuracies, model_trained, time_spent_for_training_s =\
        fit(train_dl, val_dl, model_untrained, optimizer, my_cfg.loss_fn, my_cfg.num_of_epochs, \
            scheduler=my_cfg.scheduler, apply_early_stopping = my_cfg.use_early_stopping, estop_patience=my_cfg.es_patience)
    
    
    curves = train_losses, train_accuracies, val_losses, val_accuracies, model_trained, time_spent_for_training_s
    
    
    training_curves[curve_name] = curves
    
    # Test the net
    print('\nPerformance on the test set:')
    test_loss, test_accuracy = test(model_trained, test_dl, my_cfg.loss_fn, print_loss=True)
    print('->test_loss: {:.4f}, test_accuracy: {:.4f}%'.format(test_loss, test_accuracy))


    # Store the results
    results_storer.store_results_for_plot(my_cfg,test_loss, test_accuracy, train_losses,
                                 train_accuracies, time_spent_for_training_s, val_losses, val_accuracies)

    results_storer.store_results(my_cfg, model_trained, optimizer, test_loss, test_accuracy, train_losses,
                                 train_accuracies, time_spent_for_training_s, val_losses, val_accuracies, test_dl)


plot_all_metrics(training_curves)

ConvNet1D_Adam 
-------------------------

We are not on the cluster...

Data is being loaded using MNE...


100%|██████████| 6/6 [00:32<00:00,  5.99s/it]


5418 matching events found
No baseline correction applied
...data loading with MNE was finished. 

Data is being augmented with gaussian noise...


100%|██████████| 4388/4388 [00:32<00:00, 135.01it/s]


...augmentation with gaussian noise is finished. 

cuda will be used for training of this model.
You have 2 GPUs for training!
Learning Rate:  0.001 

-> Epoch 1/15: train_loss: 1.9599, train_accuracy: 22.4567%, val_loss: 1.7981, val_accuracy: 30.7377%
-> Epoch 2/15: train_loss: 1.7235, train_accuracy: 34.2707%, val_loss: 1.7686, val_accuracy: 35.8607%
-> Epoch 3/15: train_loss: 1.6946, train_accuracy: 35.1322%, val_loss: 1.7564, val_accuracy: 34.8361%
-> Epoch 4/15: train_loss: 1.6675, train_accuracy: 36.5041%, val_loss: 1.7769, val_accuracy: 33.1967%
-> Epoch 5/15: train_loss: 1.6557, train_accuracy: 37.2744%, val_loss: 1.7362, val_accuracy: 34.0164%
-> Epoch 6/15: train_loss: 1.6372, train_accuracy: 37.7712%, val_loss: 1.7370, val_accuracy: 34.0164%
-> Epoch 7/15: train_loss: 1.6093, train_accuracy: 39.0018%, val_loss: 1.7240, val_accuracy: 35.6557%
-> Epoch 8/15: train_loss: 1.5981, train_accuracy: 39.2343%, val_loss: 1.7220, val_accuracy: 35.8607%
-> Epoch 9/15: train_loss: 1.5825